In [1]:
# Importamos las librerias necesarias
from Busquedas import *
import datetime
import pymongo
from pymongo import MongoClient
import urllib
import json
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns
import os
pd.set_option('display.max_colwidth', None)
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/gabrielarcangelbol/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# 1. Descarga de Datos

## 1.1 Conexion y descarga de datos desde Instancia de BBDD NoSQL MongoDB

In [2]:
# Autentificacion
def auth_mongo(username, password, database):
    
    # Conexion con BD NoSQL MongoDB - Creacion de BD y Colección

    username_url = urllib.parse.quote_plus(username)
    password_url = urllib.parse.quote_plus(password)

    # Fuera de GCP - Direccion Ip Externa
    # XXXX - # Por motivos de seguridad se eliminaron estas direcciones / claves / puertos
    # Dentro de GCP - Direccion Ip Interna
    # XXXX  - # Por motivos de seguridad se eliminaron estas direcciones / claves / puertos
    url = 'mongodb://%s:%s@XXXX:XXXX/'+database # Por motivos de seguridad se eliminaron estas direcciones / claves / puertos
    try: 
        client = MongoClient(url % (username_url, password_url))
        print("Conectado satisfactoriamente!!!") 
    except:   
        print("No se logro conectar a MongoDB")
    return client

# Extraer datos de mongo
def extract_mongo(client, database, collection_name):
    
    try:
        db = client[database]
        collection = db[collection_name]
        info = collection.find()
        df = pd.DataFrame(list(info))
        df.drop_duplicates('id_post', inplace=True)
        df['reactions_count'] = df['reactions_count'].astype(int)
        df['date'] = df['date'].astype('datetime64[ns]')
        print("Los datos de la coleccion %s de la base de datos %s fueron extraidos satisfactoriamente!" %(collection_name, database))
    
    except:
        print("Los datos de la coleccion %s de la base de datos %s NO fueron extraidos" %(collection_name, database))
    
    return df

# Insertar un dataframe en mongo
def insert_mongo (client, database, collection_name, df):
    
    try:
        db = client[database]
        collection = db[collection_name]
        data_dict = df.to_dict("records")
        collection.insert_many(data_dict)
        print("Los datos fueron insertados correctamente en la coleccion %s, de la base de datos %s." %(collection_name, database))
    
    except:
        print("Los datos NO fueron insertados en la coleccion %s, de la base de datos %s." %(collection_name, database))
    

In [3]:
# Definicion de parametros de descarga
database = 'GNN'
client = auth_mongo('XXXX', 'XXXX', database)    # Por motivos de seguridad se eliminaron estas direcciones / claves / puertos
clave = 'fvf'

Conectado satisfactoriamente!!!


In [4]:
# Convertir datos almacenados en MongoDB en un dataframe
# df_30days = extract_mongo(client, database, collection_name)
# df = extract_mongo(client, database, 't_fvf_progephi_merge') t_fvf_indice_influencia_v1
df = extract_mongo(client, database, 't_fvf_feb-abr_2023_tag')
df.shape

Los datos de la coleccion t_fvf_feb-abr_2023_tag de la base de datos GNN fueron extraidos satisfactoriamente!


(382, 19)

In [5]:
df

,_id,created_at,name,username,message,link,link_image,likes_count,shares_count,comments_count,type,views,user_link,id_post,date,hour,weekday,reactions_count,query
0,6474b4ebbbad6288ac9945a5,2022-02-01 15:34:29,Universitarios Sport Club,Univer_sc,¡Somos vinotinto! \n\nMucha suerte a nuestra @FVF_Oficial\n#sueñoauriazul\n#Vinotinto | . 🇻🇪,https://twitter.com/Univer_sc/status/1488536238005104653,[https://pbs.twimg.com/media/FKhX0LUXsBEukt1?format=jpg&name=small],2,2,0,carrusel,0,https://twitter.com/Univer_sc,1488536238005104653,2022-02-01,11,Martes,4,"fvf geocode:9.61501,-69.97394,328.13km"
2,6474b529bbad6288ac9945b1,2022-02-02 00:25:00,Roberts Delgado,RobertsDelgado,"Que ladilla ver un juego de @CONMEBOL no hace falta ponerse a favor de @Uruguay es injusto ir contra @FVF_Oficial, si Uruguay va ir al mundial no hace falta que le ayuden para eso juegan los 5 grandes de Suramérica y los demás que hagan un Torneo de consolación listo @fifacom_es | .",https://twitter.com/RobertsDelgado/status/1488669745796861959,[https://pbs.twimg.com/profile_images/732617891556823040/LbiilRfJ_normal.jpg],1,1,0,image,0,https://twitter.com/RobertsDelgado,1488669745796861959,2022-02-01,20,Martes,2,"fvf geocode:9.61501,-69.97394,328.13km"
5,6474b716bbad6288ac99460e,2022-02-11 21:55:30,Edgar Perez Greco,perezgreco21,"Para mí es un honor y una Gran responsabilidad ser Embajador de nuestra LigaFutve. Me compromete aun mas por el crecimiento de nuestro fútbol. Mil gracias @fvf_oficial @ligafutve en Caracas, Venezuela https://instagram.com/p/CZ2lUpiL_l8/?utm_medium=twitter… | .",https://twitter.com/perezgreco21/status/1492256004117045254,[https://pbs.twimg.com/profile_images/1619555338004340738/Spozpsfi_normal.jpg],32,7,0,image,0,https://twitter.com/perezgreco21,1492256004117045254,2022-02-11,17,Viernes,39,"fvf geocode:9.61501,-69.97394,328.13km"
8,6474b758bbad6288ac99461b,2022-02-12 16:04:29,Javier.AFEM,FutPuntoyComa,La juventud es un bello error que se corrige con los años. Siempre jóvenes. Feliz día de la Juventudnpara todos nuestros futbolistas.\n@AFEMerida\n@Teacherenlared\n@davidaraujo81\n@Ligaaaafem\n@FVF_Oficial\n@jadi__10 | .,https://twitter.com/FutPuntoyComa/status/1492530054223605764,[https://pbs.twimg.com/media/FLaILHIWUAQqpe-?format=jpg&name=small],3,1,0,carrusel,0,https://twitter.com/FutPuntoyComa,1492530054223605764,2022-02-12,12,Sábado,4,"fvf geocode:9.61501,-69.97394,328.13km"
10,6474b942bbad6288ac994677,2022-02-21 16:28:01,La Jugada Kids,Tamayito16,El es Francisco Manuel Pérez Azuaje.\n\nTiene apenas 11 Años Juega con el Deportivo Táchira.\n\nDisfruten De Este Tiro Libre.\n\n#fvf #futbolmenor #futbolbase #vinotinto | .,https://twitter.com/Tamayito16/status/1495797466997866509,[https://pbs.twimg.com/profile_images/1547089738976428033/Q5aew8KJ_normal.jpg],3,1,0,video,0,https://twitter.com/Tamayito16,1495797466997866509,2022-02-21,12,Lunes,4,"fvf geocode:9.61501,-69.97394,328.13km"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
909,64751b9cbbad6288ac995aa8,2023-04-24 01:49:03,Miggy,DelgadoMiguel97,"Aprovechando el revuelo que hay por la visita del DT de la Selección Nacional y del presidente de la FVF, quisiera saber ustedes que opinan.\n\n¿Quisieran que la Vinotinto volviese a Pueblo Nuevo? | . 🇻🇪 ⚽",https://twitter.com/DelgadoMiguel97/status/1650315888409690113,[],1,1,0,carrusel,225,https://twitter.com/DelgadoMiguel97,1650315888409690113,2023-04-23,21,Domingo,2,"fvf geocode:9.61501,-69.97394,328.13km"
911,64751bbfbbad6288ac995aaf,2023-04-24 01:57:17,Asociación Táchirense de Fútbol,asofuttachira,"Gracias por la visita al Táchira.\n\nCompartimos con ustedes el saludo de Jorge Jiménez, presidente de la Federación Venezolana de Fútbol, en su visita al estadio Polideportivo de Pueblo Nuevo, en el marco del clásico del Fútbol nacional entre el Deportivo Táchira FC y Caracas FC. | .",https://twitter.com/asofuttachira/status/1650317958244171778,[https://pbs.twimg.com/profile_images/979701107487330304/6uNi9PNd_normal.

In [ ]:
# Convertir datos almacenados en MongoDB en un dataframe
# df_30days = extract_mongo(client, database, collection_name)
# df = extract_mongo(client, database, 't_fvf_progephi_merge') t_fvf_indice_influencia_v1
df = extract_mongo(client, database, 't_fvf_feb_abr_2023_tag_2')
df.shape

## BD: GNN   
### Colecciones: 

In [5]:
# Lista de colecciones
colecciones = [
    't_fvf_busqueda',
    't_fvf_feb_abr_2023_2_tag',
    't_fvf_indice_influencia_tipo_usuario',
    't_fvf_indice_influencia_tipo_usuario_2',
    't_fvf_indice_influencia_v0_1',
    't_fvf_indice_influencia_v1',
    't_fvf_indice_influencia_v3',
    't_fvf_progephi',
    't_fvf_progephi1',
    't_fvf_progephi_merge',
    't_fvf_progephi_merge1',
    't_fvf_progephi_merge_v2',
    't_fvf_progephi_nlp',
    't_fvf_progephi_nlp_bag_of_words',
    't_fvf_progephi_nlp_bag_of_words_2',
    't_fvf_progephi_nlp_bag_of_words_3',
    't_fvf_progephi_nlp_bow_pageranks_desnormalizado',
    't_fvf_progephi_nlp_tf_idf',
    't_fvf_progephi_nlp_tf_idf_3',
    't_fvf_progephi_v2',
    't_fvf_tag'
]

In [6]:
# Definir función para aplicar la extracción a cada colección y guardar los resultados
def extract_and_save_data(client, database, colecciones):
    for coleccion in colecciones:
        df = extract_mongo(client, database, coleccion)
        print(f"Shape de {coleccion}: {df.shape}")
        df.to_csv(f"{coleccion}.csv", index = False, encoding='utf-8')  # Guardar los datos en un archivo CSV

# Llamar a la función para extraer y guardar los datos
extract_and_save_data(client, database, colecciones)

Los datos de la coleccion t_fvf_busqueda de la base de datos GNN fueron extraidos satisfactoriamente!
Shape de t_fvf_busqueda: (7848, 115)
Los datos de la coleccion t_fvf_feb_abr_2023_2_tag de la base de datos GNN fueron extraidos satisfactoriamente!
Shape de t_fvf_feb_abr_2023_2_tag: (3480, 19)
Los datos de la coleccion t_fvf_indice_influencia_tipo_usuario de la base de datos GNN fueron extraidos satisfactoriamente!
Shape de t_fvf_indice_influencia_tipo_usuario: (7074, 103)
Los datos de la coleccion t_fvf_indice_influencia_tipo_usuario_2 de la base de datos GNN fueron extraidos satisfactoriamente!
Shape de t_fvf_indice_influencia_tipo_usuario_2: (7369, 104)
Los datos de la coleccion t_fvf_indice_influencia_v0_1 de la base de datos GNN fueron extraidos satisfactoriamente!
Shape de t_fvf_indice_influencia_v0_1: (7074, 91)
Los datos de la coleccion t_fvf_indice_influencia_v1 de la base de datos GNN fueron extraidos satisfactoriamente!
Shape de t_fvf_indice_influencia_v1: (7074, 107)
Los 

In [11]:
# Definir función para abrir y mostrar las primeras 4 líneas de cada archivo CSV
def open_and_show_csv(colecciones):
    for coleccion in colecciones:
        df = pd.read_csv(f"{coleccion}.csv")
        print('----------------------------------------------------------------------------------------------------------------')
        print(f"\nPrimeras 4 líneas de {coleccion}.csv:")
        print('----------------------------------------------------------------------------------------------------------------')
        print(df.head(4))

# Llamar a la función para abrir y mostrar los archivos CSV
open_and_show_csv(colecciones)

----------------------------------------------------------------------------------------------------------------

Primeras 4 líneas de t_fvf_busqueda.csv:
----------------------------------------------------------------------------------------------------------------
                        _id           created_at              id_post  \
0  6442ae573c00c6c014a47988  2023-04-20 23:58:40  1649200946419826688   
1  6442ae573c00c6c014a47989  2023-04-20 23:57:49  1649200732233424896   
2  6442ae573c00c6c014a4798a  2023-04-20 23:56:18  1649200347645194243   
3  6442ae573c00c6c014a4798b  2023-04-20 23:55:48  1649200224773046277   

                id_str  \
0  1649200946419826688   
1  1649200732233424896   
2  1649200347645194243   
3  1649200224773046277   

                                                                                                                                                                                                                                           